In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
%%capture
!pip install pickle5
import pickle5
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
bert_69 = pickle5.load(open('/content/drive/MyDrive/Humerus/Dev Files/training_set_data_69.pkl', 'rb'))

FileNotFoundError: ignored

In [ ]:
X_data = bert_69['Avg_embed'].to_numpy()

In [ ]:
X_data = bert_69['Avg_embed'].to_numpy()
Y_data = bert_69['Bert_embed_69'].to_numpy()
xshape = X_data[0].shape
yshape = Y_data[0].shape
X_data = np.array([np.reshape(i, (xshape[0]))  for i in X_data])
Y_data = np.array([np.reshape(i, (yshape[0])) for i in Y_data])
#X_data = X_data.T
#Y_data = Y_data.T
print(X_data.shape, Y_data.shape)

(40000, 300) (40000, 69)


In [ ]:
mdl_mngr = translateEmbeddings.from_scratch(X_data.shape, Y_data.shape)

(40000, 300) (40000, 69)
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_28 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_29 (Dense)             (None, 69)                20769     
Total params: 201,369
Trainable params: 201,369
Non-trainable params: 0
_________________________________________________________________


In [ ]:
mdl_mngr.train_model(X_data, Y_data, 1000)

-------STARTING TRAINING------
-----SHAPES-------
X: (40000, 300)
Y: (40000, 69)
Epoch 1/1000
1125/1125 [==============================] - 5s 4ms/step - loss: 3.0572 - accuracy: 0.5087 - val_loss: 4.1903 - val_accuracy: 0.3257
Epoch 2/1000
1125/1125 [==============================] - 5s 4ms/step - loss: 1.5757 - accuracy: 0.6358 - val_loss: 4.1032 - val_accuracy: 0.3632
Epoch 3/1000
1125/1125 [==============================] - 5s 4ms/step - loss: 1.3233 - accuracy: 0.6678 - val_loss: 4.0756 - val_accuracy: 0.3762
Epoch 4/1000
1125/1125 [==============================] - 4s 4ms/step - loss: 1.1457 - accuracy: 0.6928 - val_loss: 4.0048 - val_accuracy: 0.3993
Epoch 5/1000
1125/1125 [==============================] - 4s 4ms/step - loss: 1.0216 - accuracy: 0.7123 - val_loss: 3.9387 - val_accuracy: 0.3955
Epoch 6/1000
1125/1125 [==============================] - 5s 4ms/step - loss: 0.9311 - accuracy: 0.7266 - val_loss: 3.9456 - val_accuracy: 0.3927
Epoch 7/1000
1125/1125 [===================

KeyboardInterrupt: ignored

In [ ]:
class translateEmbeddings:

    def __init__(self, model): 
        self.model = model

    @classmethod
    def from_scratch(cls, input_shape, output_shape):
        mdl=make_model(input_shape, output_shape)
        mdl.summary()
        return cls(mdl)

    @classmethod
    def from_file(cls, input_shape, output_shape, weight_path): 
        # get input shape
        # pass relative path strings
        mdl = make_model(input_shape, output_shape)
        mdl.load_weights(weight_path)
        return cls(mdl)

    def save_models(self, folder, name):
        # assuming path: ./mdls/l & ./mdls/r for left & right models, respectively. And left and right are of same num.
        # naming convention: {mdl_type}_num.h5
        paths = os.listdir(folder)
        try:
            paths.remove('.ipynb_checkpoint')
        except:
            pass
        if len(paths) == 0:
            n = 1
        else:
            n = max([int(re.findall("\d+", i)[0]) for i in paths]) + 1
        self.model.save(name+'_{n}.h5'.format(n=str(n)))

    def train_model(self, x_train, y_train, n_epochs):
        """
        x train: (<size of input>, <n_input>), for 300 -> 69, (300, <n_input>)
        y train: (<size of output>, <n_output>), for 300 -> 69, (69, <n_input>)
        """
        print("-------STARTING TRAINING------")

        print("-----SHAPES-------")
        print("X:", x_train.shape)
        print("Y:", y_train.shape)

        self.model.fit(x=x_train, y=y_train, \
            epochs=n_epochs, verbose=1, \
            validation_split = 0.1, shuffle=True ) 
        
    def evaluate(self, x):
        return self.model.predict(np.expand_dims(x, axis=0), batch_size=1)[0][0]


In [ ]:
def make_model(input_size, output_size):
    """
    model for dimension reduction/increase
    """
    print(input_size, output_size)
    model = keras.Sequential()
    model.add(layers.Dense(input_size[1], input_shape=(input_size[1], ), activation='tanh'))

    model.add(layers.Dense(input_size[1], activation='tanh'))

    model.add(layers.Dense(output_size[1]))
    #model.add(layers.Dense(output_size[1], input_shape=(input_size[1], )))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model